# Reading Your Geocoded Data

After assignment 3, you should have a file, which contains a latitude, longitude pair for more or less each sales record.

In [ ]:
%matplotlib notebook


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# A CSV file containing all scraped data, the result of assignment number two
complete_data = './data/boliga_all_loc.csv'

df = pd.read_csv(complete_data)
df.head()

If you want to convert the `sell_date` column into a a proper `Datetime` object, you can do so via one of the following methods.

  * Let Pandas figure out in which form your dates are stored and parse them while reading the CSV file. **OBS**: This will take a bit for the entire dataset.

In [ ]:
df = pd.read_csv(complete_data, parse_dates=['sell_date'])
df.head()

  * Let Pandas figure out in which form your dates are stored and convert a column in an existing `DataFrame`.  **OBS**: This will take a bit for the entire dataset. 

In [ ]:
pd.to_datetime(df['sell_date'])

 * Specify in which format your dates are stored and parse them while reading the CSV file.

In [ ]:
dateparse = lambda x: pd.datetime.strptime(x, '%d-%m-%Y')

df = pd.read_csv(complete_data, parse_dates=['sell_date'], date_parser=dateparse)
df.head()

# A bit more of preprocessing...

Again, we would like to have for each sales record an integer number for the zip code, so that we can use those values for boolean indexing.

In [ ]:
df['zip_nr'] = [int(el.split(' ')[0]) 
                for el in df['zip_code'].values]

Futhermore, we would like an extra column containing the sales year.

Note, in case you did not convert the `sell_date` to proper `Datetime` objects you would have to treat them as strings as in the follwing.

```python
df['sell_year'] = [int(el.split('-')[-1]) 
                   for el in df['sell_date'].values] 
```


In [ ]:
df['sell_year'] = df['sell_date'].dt.year

In [ ]:
df.head()

In [ ]:
mask = ((df['zip_nr'] < 4800) & 
        (df['sell_year'] <= 2005) & 
        (df['sell_year'] >= 2000))
df_zealand_00_05 = df[mask]
df_zealand_00_05.head()

## Scatter Plots

If you wanted to get a first impression for the locations for which we have sales records, you could plot the latitude and longitude values in a two-dimensional plot, where each location is a dot.

In [ ]:
%matplotlib notebook


x_values = df['lon']
y_values = df['lat']

plt.scatter(x_values, y_values, s=1, edgecolor='none')

Note, that these plots are not really well suited for geographical data as they just plot raw value combinations. There is no information carried about a suitable projection, see https://en.wikipedia.org/wiki/Map_projection. 

In [ ]:
%matplotlib notebook


x_values = df_zealand_00_05['lon']
y_values = df_zealand_00_05['lat']

plt.scatter(x_values, y_values, s=1, edgecolor='none')

You can modify the same plot, by just adding more values to it as in the following. 

In [ ]:
%matplotlib notebook


x_values = df['lon']
y_values = df['lat']

plt.scatter(x_values[:], y_values[:], s=1, edgecolor='none')

center_x = x_values.min() + (x_values.max() - x_values.min()) / 2
center_y = y_values.min() + (y_values.max() - y_values.min()) * 0.5

plt.scatter(center_x, center_y, s=10, c='red')

## Color


We use the Haversine Distance (https://en.wikipedia.org/wiki/Haversine_formula) to compute an array of distance values, which we use for color coding.

In [ ]:
import math


def haversine_distance(origin, destination):

    lat_orig, lon_orig = origin
    lat_dest, lon_dest = destination
    radius = 6371

    dlat = math.radians(lat_dest-lat_orig)
    dlon = math.radians(lon_dest-lon_orig)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) + math.cos(math.radians(lat_orig)) 
        * math.cos(math.radians(lat_dest)) * math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c

    return d


distances = [haversine_distance((center_x, center_y), el) 
             for el in df[['lon', 'lat']].values]

In [ ]:
%matplotlib notebook


x_values = df['lon']
y_values = df['lat']

plt.scatter(x_values, y_values, s=1, c=distances, 
            cmap=plt.cm.Blues, edgecolor='none')
plt.scatter(center_x, center_y, s=10, c='red')

In [ ]:
import math


def distance(origin, destination):

    lat_orig, lon_orig = origin
    lat_dest, lon_dest = destination

    dlat = lat_dest - lat_orig
    dlon = lon_dest - lon_orig

    d = math.sqrt(dlat ** 2 + dlon ** 2)

    return d


distances = [distance((center_x, center_y), el) 
             for el in df[['lon', 'lat']].values]

In [ ]:
%matplotlib notebook


x_values = df['lon']
y_values = df['lat']

plt.scatter(x_values[:], y_values[:], s=1, c=distances, 
            cmap=plt.cm.Blues, edgecolor='none')
plt.scatter(center_x, center_y, s=10, c='red')

## Line Plots

Line plots are a good tool for visualizing trends from one value to another. For example, let's create a line plot which compares the number of rooms to the average and median price per square meter.

In [ ]:
mask = (df_zealand_00_05['sell_year'] == 2003)
no_rooms_series = df_zealand_00_05[mask]['no_rooms']
stats = [(no,
          df_zealand_00_05[(df_zealand_00_05['no_rooms'] == no)]['price_per_sq_m'].mean(), 
          df_zealand_00_05[(df_zealand_00_05['no_rooms'] == no)]['price_per_sq_m'].median()) 
         for no in range(int(no_rooms_series.min()), int(no_rooms_series.max()))]


df_stats = pd.DataFrame(stats, columns=['no_rooms', 'mean_price', 'median_price'])
df_stats

In [ ]:
%matplotlib notebook


plt.plot(df_stats.no_rooms, df_stats.mean_price, 
         label="Mean price per sqm")
plt.plot(df_stats.no_rooms, df_stats.median_price, 
         label="Median price per sqm")
plt.legend()

What can you read out of the plot above?

### A side note on sorting values...

You can sort your `DataFrame` for example with the help of the `sort_values` method.

In [ ]:
df_stats.sort_values(by='mean_price')

In [ ]:
df_stats.sort_values(by='mean_price', inplace=True)

## Bar Plots

In [ ]:
%matplotlib notebook


from collections import Counter


mask = ((df_zealand_00_05['sell_year'] == 2005) &
        (~df_zealand_00_05['no_rooms'].isnull()))
no_rooms = df_zealand_00_05[mask]['no_rooms']

no_room_freq = list(Counter(no_rooms).values())
no_rooms = list(Counter(no_rooms).keys())

plt.bar(no_rooms, no_room_freq)

Since bar plots are most often used for plotting hsitograms, you can also resort to the `hist` function, which comutes the histogram in the background for you.

In [ ]:
%matplotlib notebook


mask = ((df_zealand_00_05['sell_year'] == 2005) &
        (~df_zealand_00_05['price_per_sq_m'].isnull()))
prices = df_zealand_00_05[mask]['price_per_sq_m']
binwidth = 1000
plt.hist(prices, bins=range(int(prices.min()), 
                            int(prices.max() + binwidth), binwidth))

# Your Turn!

![](https://camo.githubusercontent.com/320b4791da998fd94e34ad4a85d44d8d5a581ca4/68747470733a2f2f732d6d656469612d63616368652d616b302e70696e696d672e636f6d2f6f726967696e616c732f39662f37332f65332f39663733653366386139353864626530336230663736663838313161353461312e676966)


We get all steps of this weeks assignment to run!

You will need some of the files for next week. In case you have a working solution we try to make your code more short, effective, good looking, and easily runable.

# Plotting Geographic Data with `Basemap`s


If you want to plot your data, which contains geo-locations with with a proper projection onto a plane, then you might want to have a look on the `Basemap` class.

In [ ]:
%matplotlib notebook


import warnings
warnings.filterwarnings('ignore')
from mpl_toolkits.basemap import Basemap


plt.figure(figsize=(5, 5))
m = Basemap(projection='ortho', resolution=None, lat_0=50, lon_0=10)
m.bluemarble(scale=0.5)

Lambert conformal conic projection https://en.wikipedia.org/wiki/Lambert_conformal_conic_projection

In [ ]:
%matplotlib notebook


fig = plt.figure(figsize=(8, 8))
m = Basemap(projection='lcc', resolution=None,
            width=5000000, height=5000000, 
            lat_0=55, lon_0=10,)
m.etopo(scale=1.0, alpha=0.5)

# Map (long, lat) to (x, y) for plotting
coords = {'København': (55.676111, 12.568333), 
          'Odense': (55.395833, 10.388611), 
          'Aalborg': (57.05, 9.916667)}

for city, coord in coords.items():
    y_adjust = 100000 if city is 'Odense' else 0
    
    x, y = m(coord[1], coord[0])
    plt.plot(x, y, 'ok', markersize=5)
    
    plt.text(x + 50000, y - y_adjust, city, fontsize=8)


Mercator projection https://en.wikipedia.org/wiki/Mercator_projection

In [ ]:
mask = ((~df_zealand_00_05.lat.isnull()) & 
        (~df_zealand_00_05.lon.isnull()) & 
        (df_zealand_00_05.no_rooms >= 5))
df_zealand_00_05_large = df_zealand_00_05[mask]

In [ ]:
%matplotlib inline


# create new figure, axes instances.
fig = plt.figure()
ax = fig.add_axes([x_values.min(), y_values.min(), 
                   x_values.max(), y_values.max()])

# setup mercator map projection.
m = Basemap(llcrnrlon=7., llcrnrlat=54., 
            urcrnrlon=16., urcrnrlat=58.,
            rsphere=(6378137.00, 6356752.3142),
            resolution='h', projection='merc',
            lat_0=40., lon_0=-20., lat_ts=20.)

m.drawcoastlines()
m.fillcontinents(zorder=0)
m.scatter(df_zealand_00_05_large.lon.values, 
          df_zealand_00_05_large.lat.values, 
          3, marker='o', latlon=True)

# draw parallels
m.drawparallels(np.arange(10, 90,  1), 
                labels=[1, 1, 0, 1])
# draw meridians
m.drawmeridians(np.arange(-180, 180, 1), 
                labels=[1, 1, 0, 1])
ax.set_title('Big Flats on Zealand')
plt.show()

# Plotting Geodata with `folium`

The above `Basemap` plots are nice. However, they are static images. In case you want to create plots for the web, you might want to have a look on the `folium` module, see http://folium.readthedocs.io/en/latest/, which is a Python wrapper for `Leaflet` http://leafletjs.com.

In [ ]:
from datetime import date
from datetime import datetime


mask = ((~df_zealand_00_05.lat.isnull()) & 
        (~df_zealand_00_05.lon.isnull()) & 
        (df_zealand_00_05.no_rooms >= 6) &
        (df_zealand_00_05.sell_date >= datetime(2003, 3, 24)) & 
        (df_zealand_00_05.sell_date <= datetime(2003, 9, 15))
       )
df_zealand_00_05_large = df_zealand_00_05[mask]
df_zealand_00_05_large

In [ ]:
import folium


my_map = folium.Map(location=[55.88207495748612, 10.636574309440173], zoom_start=6)

for coords in zip(df_zealand_00_05_large.lon.values, 
                  df_zealand_00_05_large.lat.values):
    folium.CircleMarker(location=[coords[1], coords[0]], radius=2).add_to(my_map)
my_map.save('data/large_flat_trades.html')
my_map

In [ ]:
%%bash

ls -ltrh data/large_flat_trades.html

In [ ]:
%%bash

tail -30 data/large_flat_trades.html

### GeoJson File for Danish Postnumre Data

I got it from https://github.com/ok-dk/dagi, which says that the data is originally from Geodatastyrelsen and Danske Kommuner.

In [ ]:
%%bash

wget https://raw.githubusercontent.com/ok-dk/dagi/master/geojson/postnumre.geojson

In [ ]:
import os
import json
import folium
import requests


def get_geojson(url):
    local_file = os.path.basename(url)
    if os.path.isfile(local_file):
        # if file exists locally load it
        with open(local_file, encoding='utf-8') as f: 
            geo_json = json.load(f)
    else:
        # otherwise read it from remote
        response = requests.get(url)
        geo_json = response.json()
    return geo_json


def get_city_location(city='Copenhagen'):
    url_googleapi = "https://maps.googleapis.com/maps/api/geocode/json"
    r = requests.get(url_googleapi, params={'sensor': 'false', 'address': city})
    results = r.json()['results']
    
    location = results[0]['geometry']['location']
    lat, lon = location['lat'], location['lng']
    return lat, lon


url = "https://raw.githubusercontent.com/ok-dk/dagi/master/geojson/postnumre.geojson"
geo_json = get_geojson(url)

lat, lon = get_city_location(city='Helsingør')
map_osm = folium.Map(location=(lat, lon), zoom_start=6)
folium.GeoJson(geo_json, name='geojson').add_to(map_osm)
map_osm.save('./osm2.html')
map_osm

## Filtering a GeoJSON File

In [ ]:
%%bash

ls -ltrh postnumre.geojson

In [ ]:
%%bash

cat postnumre.geojson

In [ ]:
import json


geo_json_zealand = {}

geo_json_zealand['type'] = geo_json['type']
geo_json_zealand['crs'] = geo_json['crs']
geo_json_zealand['features'] = []

for feature in geo_json['features']:
    if '-' in feature['properties']['POSTNR_TXT']:
        zip_code = int(feature['properties']['POSTNR_TXT'].split('-')[-1])
    else:
        zip_code = int(feature['properties']['POSTNR_TXT'])
    if zip_code < 4800:
        # keep this feature
        geo_json_zealand['features'].append(feature)

        
with open('./data/zip_areas_zealand.geojson', 'w', encoding='utf-8') as f:
    json.dump(geo_json_zealand, f)

In [ ]:
lat, lon = get_city_location(city='Helsingør')
map_osm = folium.Map(location=(lat, lon) , zoom_start=7)
folium.GeoJson(geo_json_zealand, name='geojson').add_to(map_osm)
map_osm

We add a new column with an integer value for the zip code, so that we can easily filter out data for various  regions.

In [ ]:
zip_strings = []
# ['1800-1899', '1500-1799', '1000-1499']
for z in df_zealand_00_05['zip_nr'].values:
    if 1000 <= z <= 1499:
        zip_str = '1000-1499'
    elif 1500 <= z <= 1799:
        zip_str = '1500-1799'
    elif 1800 <= z <= 1899:
        zip_str = '1800-1899'
    else:
        zip_str = str(z)
    zip_strings.append(zip_str)

#df_zealand_00_05['zip_strings'] = zip_strings
df_zealand_00_05 = df_zealand_00_05.assign(zip_strings=zip_strings)

In [ ]:
zip_codes_str = df_zealand_00_05['zip_strings'].unique()
year_of_interest = 2005
avg_sq_m_prices_per_zip = [df_zealand_00_05[(df_zealand_00_05['zip_strings'] == z) & 
                                            (df_zealand_00_05['sell_year'] == year_of_interest)]['price_per_sq_m'].mean() 
                           for z in zip_codes_str]

In [ ]:
len(avg_sq_m_prices_per_zip)

In [ ]:
data = {'avg_sq_m_prices_per_zip': avg_sq_m_prices_per_zip,
        'zip_code': zip_codes_str}

df_avg_price = pd.DataFrame(data)
df_avg_price.head()

In [ ]:
geo_json_zealand

In [ ]:
import folium


#Let Folium determine the scale
my_map = folium.Map(location=[55.712401893184655, 12.36683945946928], 
                    zoom_start=7)
my_map.choropleth(geo_path='data/zip_areas_zealand.geojson', 
                  data=df_avg_price,
                  columns=['zip_code', 'avg_sq_m_prices_per_zip'],
                  key_on='feature.properties.POSTNR_TXT',
                  fill_color='YlOrRd', fill_opacity=0.7, 
                  line_opacity=0.2,
                  legend_name='Average Price per square meter')
    
my_map